#Instalando as bibliotecas

In [31]:
pip install pypdf2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
pip install reportlab

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import re
import PyPDF2

#Extaindo o texto

In [34]:
#extração do texto usando a função PdfReader e separando o conteúdo das páginas
#passa-se o nome do arquivo pdf
pdf = PyPDF2.PdfReader("edital-conc-077-2023-site.pdf")
number_of_pages = len(pdf.pages)
pages = []
for page in range(number_of_pages):
  page = pdf.pages[page]
  page_text = page.extract_text()
  pages.append(page_text)

In [35]:
#colocando todo o texto em uma unica string
total_text = ""
for page in pages:
  total_text += page

#Encontrando o tipo de edital


In [36]:
#funçao para encontrar as linhas em que o tipo de edital está presente
def encontrar_tipos_de_edital(texto, tipos_possiveis):
    linhas_encontradas = []
    linhas = texto.split("\n")

    for idx, linha in enumerate(linhas):
        for tipo in tipos_possiveis:
            if tipo in linha.lower():
                linhas_encontradas.append((linha))
                break

    return linhas_encontradas

#funçao para encontrar a string do tipo de edital
def encontrar_nome_tipo(texto, tipos_possiveis):
    for tipo in tipos_possiveis:
        if tipo in texto.lower():
            return tipo

#lista dos tipos de editais possiveis
tipos_possiveis = ["concorrência", "carta convite", "tomada de preços", "pregão eletrônico", "pregão presencial", "dispensa de licitação"]

#chamada das funções passando a primeira pagina e a lista de tipos como parametro
linhas_encontradas = encontrar_tipos_de_edital(pages[0], tipos_possiveis)
tipo = encontrar_nome_tipo(pages[0], tipos_possiveis)

#laço de repetição que percorre as linhas encontradas que tinham o tipo do edital
#procura o indice do tipo para porcurar o numero dele e retornar o tipo_edital
for linha in linhas_encontradas:
  linha_corrigida = " ".join(linha.split())
  if (tipo.strip() + " n") in linha_corrigida.lower():
    a = linha_corrigida.lower().find(tipo)
    tipo_edital = linha_corrigida[a::]
print(tipo_edital)

CONCORRÊNCIA Nº. 077/2023


#Encontrando o objeto

In [37]:
# padrao da expressão regular
padrao_objeto = r"contratação[^.]*\."

#cria uma lista com as correspondecias a partir do padrao criado
correspondencias = re.findall(padrao_objeto, pages[0], re.IGNORECASE)

if correspondencias: #se a lista nao estiver vazia, correspondencia recebe o primeiro elemento da lista
    correspondencia = correspondencias[0]

    #remove quebras de linha e espaços em branco
    correspondencia_sem_quebras = correspondencia.replace("\n", "").strip()

    #encontra a posição de "PB" na string
    posicao_pb = correspondencia_sem_quebras.find("PB")
    
    #verifica se "PB" foi encontrado e salva o conteudo em objeto
    if posicao_pb != -1:
        objeto = correspondencia_sem_quebras[:posicao_pb+2]
        print(objeto)
else:
    objeto = "Nenhum objeto encontrado."
    print(objeto)

#Encontrando a cidade

In [38]:
#criando o padrao para achar o nome da cidade
padrao_cidade = r"\b[A-Z][a-zA-Záéíóúãõâêîôûàèìòù\s]+/[A-Z]{2}\b"

#busca das apariçoes desse padrao
aparicoes = re.findall(padrao_cidade, total_text)

#laço de repetiçao para retirar os espaços extras da string
aparicoes_corrigidas = []
for a in aparicoes:
  a = " ".join(a.split())
  aparicoes_corrigidas.append(a)

#verifica se a lista está vazia
if not aparicoes_corrigidas:
  nome_cidade = "Não encontrado"

#se o tamanho da lista for igual a 1, procura-se o " de ",
#pois geralmente o nome da cidade esta precedido por "prefeitura municipal de" ou "cidade de"
#e salva o nome da cidade
if (len(aparicoes_corrigidas) == 1):
  a = aparicoes_corrigidas[0].lower().find(" de ")
  nome_cidade = aparicoes_corrigidas[0][a+4::]

#se todas as aparicoes forem iguais, salva o primeiro elemento da lista
elif all(elem == aparicoes_corrigidas[0] for elem in aparicoes_corrigidas):
    nome_cidade = aparicoes_corrigidas[0]

#se estiver em alguma outra posição da lista, procura se o " de ", com o mesmo proposito anterior
else:
  for aparicao in aparicoes_corrigidas:
    if " de " in aparicao.lower():
      a = aparicao.lower().find(" de ")
      aparicao_corrigida = aparicao[a+4::]
      nome_cidade = aparicao_corrigida
      break
print(nome_cidade)


João Pessoa/PB


#Encontrando o valor

In [39]:
#cria o padrao para achar o valor
padrao_valor = r'R\$ [\d.,]+\s+\([^)]+\)'

#busca as apariçoes a partir do padrao
aparicoes = re.search(padrao_valor, total_text)

#verifica se a lista esta vazia
if not aparicoes:
  valor = "Valor nao declarado"

#pega a primeira aparição do valor e troca o "new line" por um espaço vazio
else:
  valor = aparicoes[0].replace("\n", "")
  valor = ' '.join(valor.split())

print(valor)

R$ 5.813.431,65 (Cinco milhões, oitocentos e treze mil, quatrocentos e trinta e um reais e sessenta e cinco centavos)


#Encontrando os anexos

In [40]:
#funçao para extrair os anexos
def extrai_anexos(pdf_text):
  #cria o padrao para os anexos
  padrao_anexo = r'Anexo\s+(\w+)\s*[-–]\s*([^;]+)'

  #separa o texto em linhas
  lines = pdf_text.split('\n')

  #laço de repetiçao pegar o nome dos anexos
  anexos = []
  for line in lines:
    match = re.search(padrao_anexo, line, re.IGNORECASE)
    if match:
      nome_anexo = match.group(2).strip()
      anexos.append(nome_anexo)
  return anexos

anexos = extrai_anexos(total_text)
for anexo in anexos:
    print(anexo)

Minuta de contrato
Modelo de proposta
Modelo de Declaração relativa à proibição do trabalho do menor (Lei nº 9.854/99)
Modelo de Declaração de que recebeu to dos os documentos pertinentes à licitação e de que tomou conhecimento
Modelo de Declaração de Microempresa, de empresa de pequeno porte, ou de coop erativa enquadrada no art. 34 da
Modelo do Quadro de Composição da Taxa de BDI
Modelo do Quadro de Composição da Taxa de Encargos Sociais
Modelo de credenciamento
Modelo de declaração comprometendo -se a empregar 10%, em relação ao total dos empregados, da mão de obra
Projeto Básico:
Modelo de declaração comprometendo -se a empregar 5% do total de vagas existentes na contratação da obra desta
Modelo de declaração de adoção da taxa de BDI do órg ão licitante. Substitui o ANEXO 6
Declaração de superveniência de fato impeditivo de habilitação para as licitantes cadastradas
Quadro de Pessoal Técnico
Modelo de Declaração de não relação de parentesco vedada pelo Inciso III do Art. 1º da Lei 

In [41]:
#concatena os elementos de anexo em uma string, separando-os por ";"
string_anexos = "; ".join([s for s in anexos])

#Gerando o resumo


In [42]:
resumo = f"Edital de licitação na modalidade {tipo_edital}, cujo objeto trata-se da {objeto},\nno município de {nome_cidade}. Acompanham o edital os seguintes documentos: planilha orçamentária, com valor global de {valor},\n{string_anexos}."
print(resumo)

Edital de licitação na modalidade CONCORRÊNCIA Nº. 077/2023, cujo objeto trata-se da Contratação de uma empresa de engenharia para a Execução de Serviços de Pavimentação com Capeamento Asfáltico (CBUQ) em 43 Ruas/Avenidas, nos bairros: Aeroclube, Água Fria, Alto do Céu, Bairro dos Estados, Bairro dos Ipês, Brisamar, Castelo Branco, Colibris, Costa e Silva, Cristo Redentor, Cruz das Armas, Cuiá e Distrito Industrial – Lote 01,  na Cidade de João Pessoa/PB,
no município de João Pessoa/PB. Acompanham o edital os seguintes documentos: planilha orçamentária, com valor global de R$ 5.813.431,65 (Cinco milhões, oitocentos e treze mil, quatrocentos e trinta e um reais e sessenta e cinco centavos),
Minuta de contrato; Modelo de proposta; Modelo de Declaração relativa à proibição do trabalho do menor (Lei nº 9.854/99); Modelo de Declaração de que recebeu to dos os documentos pertinentes à licitação e de que tomou conhecimento; Modelo de Declaração de Microempresa, de empresa de pequeno porte, ou

#Extra


Gerado completamente pelo ChatGPT, apenas por curiosidade

In [43]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def generate_pdf(file_name, content):
    c = canvas.Canvas(file_name, pagesize=letter)
    width, height = letter

    words = content.split(' ')
    y_position = height - 100

    current_line = ""
    for word in words:
        if len(current_line) + len(word) <= 80:  # Verifica se a próxima palavra cabe na linha atual
            if current_line:
                current_line += " " + word
            else:
                current_line = word
        else:
            c.drawString(100, y_position, current_line)
            y_position -= 15
            current_line = word

    if current_line:
        c.drawString(100, y_position, current_line)

    c.showPage()
    c.save()

generate_pdf(f"Resumo.pdf", resumo)